In [1]:
from book.mat import Mat
from book.vec import Vec
from book.vecutil import list2vec
from book.matutil import *
from book.GF2 import one, zero

In [10]:
# Task 4.14.1
def togf2(d):
    if d not in ["1", "0"]:
        raise
    return one if d == "1" else zero

rows = "1011 1101 0001 1110 0010 0100 1000".split(' ')

G = listlist2mat([[togf2(ch) for ch in r] for r in rows])

def fn_g(v_x):
    return G*v_x

[
    G,
    [G[0, j] for j in G.D[1]],
    [G[4, j] for j in G.D[1]]
]

[Mat(({0, 1, 2, 3, 4, 5, 6}, {0, 1, 2, 3}), {(0, 0): one, (0, 1): 0, (0, 2): one, (0, 3): one, (1, 0): one, (1, 1): one, (1, 2): 0, (1, 3): one, (2, 0): 0, (2, 1): 0, (2, 2): 0, (2, 3): one, (3, 0): one, (3, 1): one, (3, 2): one, (3, 3): 0, (4, 0): 0, (4, 1): 0, (4, 2): one, (4, 3): 0, (5, 0): 0, (5, 1): one, (5, 2): 0, (5, 3): 0, (6, 0): one, (6, 1): 0, (6, 2): 0, (6, 3): 0}),
 [one, 0, one, one],
 [0, 0, one, 0]]

In [11]:
# Task 4.14.2
p = list2vec([1, 0, 0, 1])
print(fn_g(p))


 0 1   2   3 4 5   6
--------------------
 0 0 one one 0 0 one


In [14]:
from sympy.abc import a, b, c, d
from sympy import Matrix
# Task 4.14.3
# Option 1: RGc -> c, so the R could be G^-1

# Option 2: the encoded_c already has the original message embeded inside
G_sym = Matrix([[int(ch) for ch in r] for r in rows])
c = Matrix([[a, b, c, d]]).transpose()
display(G_sym * c)

# We could manually construct such as an R_sym
R_sym = Matrix.zeros(4, 7)
R_sym[3, 2] = 1 # pick d
R_sym[2, 4] = 1
R_sym[1, 5] = 1
R_sym[0, 6] = 1
[display(e) for e in 
[
    R_sym,
    R_sym * (G_sym * c),
]]

Matrix([
[a + c + d],
[a + b + d],
[        d],
[a + b + c],
[        c],
[        b],
[        a]])

Matrix([
[0, 0, 0, 0, 0, 0, 1],
[0, 0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 1, 0, 0],
[0, 0, 1, 0, 0, 0, 0]])

Matrix([
[a],
[b],
[c],
[d]])

[[0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1, 0, 0],
 [0, 0, 1, 0, 0, 0, 0]]

[None, None, None]

In [5]:
rows = "0001111 0110011 1010101".split(' ')
H = listlist2mat([[togf2(ch) for ch in r] for r in rows])
H_sym = Matrix([[int(ch) for ch in r] for r in rows])

# Error syndrome matrix
def fn_h(v_y):
    return H*v_y

[
    H*G,
    H_sym*G_sym,  # 2 represents 0 in GF(2)
]

[Mat(({0, 1, 2}, {0, 1, 2, 3}), {(0, 0): 0, (0, 1): 0, (0, 2): 0, (0, 3): 0, (1, 0): 0, (1, 1): 0, (1, 2): 0, (1, 3): 0, (2, 0): 0, (2, 1): 0, (2, 2): 0, (2, 3): 0}),
 Matrix([
 [2, 2, 2, 0],
 [2, 2, 0, 2],
 [2, 0, 2, 2]])]

In [72]:
# 4.14.5 find_error
# insight: the column are all unique
e = list2vec([0, 0, 0, 1, 0, 0, 0])
print(H*e)

def find_error(error_syndrome):
    """vec<3> -> vec<7>"""
    cols = mat2coldict(H)
    errors = [i for i, col in cols.items() if col == error_syndrome]
    if errors:
        error_i = errors[0]
        e = [zero] * 7
        e[error_i] = one
    else:
        e = [zero] * 7

    return list2vec(e)

print(find_error(list2vec([one, one, zero])))


   0 1 2
--------
 one 0 0

 0 1 2 3 4   5 6
----------------
 0 0 0 0 0 one 0


In [20]:
# 4.14.6
c_hat = list2vec([togf2(ch) for ch in list("1011011")])
R = listlist2mat(R_sym.tolist())
def correct(msg):
    error = find_error(H*msg)
    return R*(msg + error)

[
    c_hat,
    correct(c_hat),
]

[Vec({0, 1, 2, 3, 4, 5, 6},{0: one, 1: 0, 2: one, 3: one, 4: 0, 5: one, 6: one}),
 Vec({0, 1, 2, 3},{0: 0, 1: one, 2: 0, 3: one})]

In [35]:
# 4.14.7
def find_error_matrix(S):
    return coldict2mat({i: find_error(col) for i, col in mat2coldict(S).items()})

find_error_matrix(listlist2mat([[one, zero], [one, zero], [one, one]]))

Mat(({0, 1, 2, 3, 4, 5, 6}, {0, 1}), {(0, 0): 0, (1, 0): 0, (2, 0): 0, (3, 0): 0, (4, 0): 0, (5, 0): 0, (6, 0): one, (0, 1): one, (1, 1): 0, (2, 1): 0, (3, 1): 0, (4, 1): 0, (5, 1): 0, (6, 1): 0})

In [40]:
# Task 4.14.8
from book.bitutil import *
import inspect
code = inspect.getsource(correct)

bits2str(
    str2bits(code)
)

'def correct(msg):\n    error = find_error(H*msg)\n    return R*(msg + error)\n'

In [46]:
# Task 4.14.9/10
bits2str(
    mat2bits(
        bits2mat(
            str2bits(
                code))))

'def correct(msg):\n    error = find_error(H*msg)\n    return R*(msg + error)\n'

In [48]:
# 4.14.11
P = bits2mat(str2bits(code)) 
E = noise(P, 0.02)
bits2str(mat2bits(P + E))

'def correct(msg):\x0e \xa0  error = find_error(H*msg)\n$`  vmturn Rª(mwg + error)\n'

In [59]:
# 4.14.12
C = G * P
[
    # before
    len(P.D[0]) * len(P.D[1]),
    # after
    len(C.D[0]) * len(C.D[1]),
]

[600, 1050]

In [63]:
# 4.14.13
C_TILDE = C + noise(C, 0.02)
bits2str(mat2bits(R*C_TILDE))

'def gorrect(irg(:\n (  error = findOerrkr(H*msw)\n   (return R*(Msg + error!\x8a'

In [74]:
# 4.14.14/15
def correct(A):
    return coldict2mat({i: col + find_error(H*col) for i, col in mat2coldict(A).items()})
bits2str(mat2bits(R*correct(C_TILDE)))

'def correct(mvg):\n    error = find_error(H*msg)\n    return R*(msg + error)\n'

In [97]:
def char_diff(str1, str2):
    diff = 0
    for i in range(min(len(str1), len(str2))):
        if str1[i] != str2[i]:
            diff += 1
    diff += abs(len(str1) - len(str2))
    return diff
    
def exp(N=10, p=0.02):
    diff = 0.0
    for _ in range(N):
        P = bits2mat(str2bits(code))
        C = G * P
        C_TILDE = C + noise(C, p)
        bits2str(mat2bits(R*C_TILDE))
        received = bits2str(mat2bits(R*correct(C_TILDE)))
        diff += char_diff(code, received)
    return diff/N

[exp(p=p/100) for p in range(1, 11)]


[0.1, 1.6, 1.5, 3.3, 6.4, 10.2, 12.4, 12.5, 17.3, 19.3]